# CLEAR System Demo Notebook

## NAVER AiRS 기반 뉴스 추천 및 주식 영향 분석 시스템

이 노트북은 NAVER의 AiRS 알고리즘을 기반으로 한 뉴스 추천 및 주식 영향 분석 시스템인 CLEAR의 기능을 시연합니다. 이 시스템은 뉴스 클러스터링, 추천, 그리고 주식 가격에 미치는 영향을 분석하는 기능을 포함합니다.

This notebook demonstrates the functionality of CLEAR, a news recommendation and stock impact analysis system based on NAVER's AiRS algorithm. The system includes news clustering, recommendation, and analysis of impact on stock prices.

## 1. 시스템 개요 (System Overview)

CLEAR 시스템은 다음과 같은 주요 구성 요소로 이루어져 있습니다:
1. 데이터 처리 (뉴스 및 주식 데이터)
2. 텍스트 전처리
3. 뉴스 벡터화
4. 뉴스 클러스터링
5. 주식 영향 분석
6. 뉴스 추천

The CLEAR system consists of the following main components:
1. Data processing (news and stock data)
2. Text preprocessing
3. News vectorization
4. News clustering
5. Stock impact analysis
6. News recommendation

In [27]:
# 필요한 라이브러리 임포트
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings

# 경고 메시지 숨기기
warnings.filterwarnings('ignore')

# 시스템 경로에 src 디렉토리 추가
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), 'src')))

# 그래프 스타일 설정
plt.style.use('ggplot')
sns.set(style="whitegrid")

print("CLEAR 시스템 데모 환경이 준비되었습니다.")
print("CLEAR system demo environment is ready.")

CLEAR 시스템 데모 환경이 준비되었습니다.
CLEAR system demo environment is ready.


## 2. 데이터 로딩 (Data Loading)

먼저 뉴스 데이터와 주식 데이터를 로드합니다. 이 데이터는 시스템의 기본 입력으로 사용됩니다.

First, we load the news data and stock data. This data is used as the basic input for the system.

In [28]:
# 뉴스 데이터 로드
news_data_path = os.path.join(os.getcwd(), 'data', 'news', 'yna_005930_all.csv')
news_df = pd.read_csv(news_data_path)

# 주식 데이터 로드
stock_data_path = os.path.join(os.getcwd(), 'data', 'stock', 'stockprice_005930.csv')
stock_df = pd.read_csv(stock_data_path)

# 데이터 확인
print(f"뉴스 데이터 샘플 수: {len(news_df)}")
print(f"주식 데이터 샘플 수: {len(stock_df)}")

# 뉴스 데이터 샘플 확인
print("\n뉴스 데이터 샘플:")
news_df.head()

뉴스 데이터 샘플 수: 32208
주식 데이터 샘플 수: 2856

뉴스 데이터 샘플:


,Title,Date,Press,Link,Body,Emotion,Num_comment,AI Summary
0,"세계 2위 휴대전화 시장 인도, 이제 중·고가 제품에 눈 돌린다",20190101 15:36,yna,https://www.yna.co.kr/view/AKR20190101051400077,"김영현기자\n구독\n""저가에서 고급 브랜드로 관심 옮겨…5년간 4배 성장 예상""\n...","{'GOOD': '0', 'SAD': '0', 'ANGRY': '0', 'NEXT'...",NaN,NaN
1,[부고] 홍현칠(삼성전자 서남아총괄)씨 부친상,20190101 14:53,yna,https://www.yna.co.kr/view/AKR20190101048900077,"김영현기자\n구독\n▲ 홍선기씨 별세, 홍현구·현칠(삼성전자 서남아총괄 부사장)·현...","{'GOOD': '0', 'SAD': '0', 'ANGRY': '0', 'NEXT'...",NaN,NaN
2,"SK그룹, CES서 신사업 기회 모색…경영진 총출동",20190101 09:00,yna,https://www.yna.co.kr/view/AKR20181231260700003,배영경기자\n구독\n(서울=연합뉴스) 배영경 기자 = SK이노베이션·SK텔레콤·SK...,"{'GOOD': '0', 'SAD': '0', 'ANGRY': '0', 'NEXT'...",NaN,NaN
3,'알박기 노조'에 미행까지…삼성에버랜드 노조방해 13명 기소,20190101 09:00,yna,https://www.yna.co.kr/view/AKR20181231271300004,김계연기자\n구독\n교섭창구 단일화 악용해 어용노조 결성…설립신고서 대리작성\n조합...,"{'GOOD': '0', 'SAD': '0', 'ANGRY': '0', 'NEXT'...",NaN,NaN
4,CES 달굴 모바일은…LG·소니 중저가폰에 로욜 폴더블폰도,20190101 08:10,yna,https://www.yna.co.kr/view/AKR20181231240500017,채새롬기자\n구독\n삼성전자는 갤노트9·갤럭시홈 전시…갤럭시홈 출시 일정은 미정\n...,"{'GOOD': '0', 'SAD': '0', 'ANGRY': '0', 'NEXT'...",NaN,NaN


In [32]:
# 주식 데이터 샘플 확인
print("주식 데이터 샘플:")
stock_df.head(20)

주식 데이터 샘플:


,Date,Time,Start,High,Low,End,Volume
0,20250305,0,55100,55400,54200,54300,6367338
1,20250304,0,53900,55000,53800,54500,18553933
2,20250228,0,55400,55700,54500,54500,28036698
3,20250227,0,56500,57100,56200,56300,14975356
4,20250226,0,57000,57100,56100,56600,18117091
5,20250225,0,56600,57800,56500,57200,14625181
6,20250224,0,57300,57700,57200,57300,14138471
7,20250221,0,58400,58500,57100,58200,22198428
8,20250220,0,59100,59100,58100,58400,21589059
9,20250219,0,57400,58900,57300,58700,25395151


### 2.1 데이터 전처리 (Data Preprocessing)

뉴스 데이터와 주식 데이터를 분석에 적합한 형태로 전처리합니다.

We preprocess the news data and stock data into a form suitable for analysis.

In [30]:
# 날짜 형식 변환 함수
def convert_date_format(date_str):
    try:
        # 예: '20250101 18:56' -> '2025-01-01 18:56:00'
        date_parts = date_str.split(' ')
        date = date_parts[0]
        time = date_parts[1] if len(date_parts) > 1 else '00:00'
        
        year = date[:4]
        month = date[4:6]
        day = date[6:8]
        
        return f"{year}-{month}-{day} {time}:00"
    except Exception as e:
        print(f"날짜 변환 오류: {e} - {date_str}")
        return None

# 뉴스 데이터 전처리
news_df['Date'] = news_df['Date'].apply(convert_date_format)
news_df['datetime'] = pd.to_datetime(news_df['Date'])
news_df = news_df.sort_values('datetime')

# 주식 데이터 전처리
stock_df['datetime'] = pd.to_datetime(stock_df['Date'] + ' ' + stock_df['Time'])
stock_df = stock_df.sort_values('datetime')

# 주식 가격 변동 계산
stock_df['price_change'] = stock_df['End'] - stock_df['Start']
stock_df['price_change_pct'] = (stock_df['price_change'] / stock_df['Start']) * 100

# 전처리된 데이터 확인
print("전처리된 뉴스 데이터:")
news_df[['Title', 'datetime', 'Press']].head()

UFuncTypeError: ufunc 'add' did not contain a loop with signature matching types (dtype('int64'), dtype('<U1')) -> None

In [31]:
# 전처리된 주식 데이터 확인
print("전처리된 주식 데이터:")
stock_df[['datetime', 'Start', 'End', 'price_change', 'price_change_pct']].head()

전처리된 주식 데이터:


KeyError: "['datetime', 'price_change', 'price_change_pct'] not in index"

## 3. 텍스트 전처리 (Text Preprocessing)

뉴스 텍스트를 분석에 적합한 형태로 전처리합니다. 이 과정에는 불용어 제거, 토큰화 등이 포함됩니다.

We preprocess the news text into a form suitable for analysis. This process includes removing stopwords, tokenization, etc.

In [ ]:
# 텍스트 전처리 모듈 임포트
from src.data.text_preprocessor import TextPreprocessor

# 텍스트 전처리기 초기화
preprocessor = TextPreprocessor(
    language='korean',
    remove_stopwords=True,
    remove_punctuation=True,
    remove_numbers=False,
    use_mecab=True
)

# 샘플 텍스트 전처리
sample_text = news_df['Title'].iloc[0] + ". " + news_df['Body'].iloc[0][:200]
processed_text = preprocessor.preprocess_text(sample_text)

print("원본 텍스트:")
print(sample_text)
print("\n전처리된 텍스트:")
print(processed_text)

In [ ]:
# 뉴스 데이터 전체 전처리
print("뉴스 데이터 전처리 중...")

# 제목 전처리
news_df['processed_title'] = news_df['Title'].apply(preprocessor.preprocess_text)

# 본문 전처리 (시간 절약을 위해 일부만 처리)
news_df['processed_body'] = news_df['Body'].apply(
    lambda x: preprocessor.preprocess_text(x[:500]) if isinstance(x, str) else ""
)

print("전처리 완료!")

# 전처리 결과 확인
news_df[['Title', 'processed_title', 'processed_body']].head(2)

## 4. 뉴스 벡터화 (News Vectorization)

전처리된 뉴스 텍스트를 벡터로 변환합니다. 여러 벡터화 방법을 비교해 보겠습니다.

We convert the preprocessed news text into vectors. Let's compare several vectorization methods.

In [ ]:
# 뉴스 벡터화 모듈 임포트
from src.models.news_vectorizer import NewsVectorizer

# 기본 벡터화 방법 (TF-IDF)
tfidf_vectorizer = NewsVectorizer(
    method='tfidf',
    title_weight=0.7,
    content_weight=0.3,
    max_features=100
)

# Word2Vec 벡터화 방법
w2v_vectorizer = NewsVectorizer(
    method='word2vec',
    title_weight=0.7,
    content_weight=0.3,
    vector_size=100
)

# 벡터화 수행
print("TF-IDF 벡터화 수행 중...")
tfidf_vectors = tfidf_vectorizer.vectorize_dataframe(
    news_df,
    title_col='processed_title',
    content_col='processed_body'
)

print("Word2Vec 벡터화 수행 중...")
w2v_vectors = w2v_vectorizer.vectorize_dataframe(
    news_df,
    title_col='processed_title',
    content_col='processed_body'
)

print(f"TF-IDF 벡터 형태: {tfidf_vectors.shape}")
print(f"Word2Vec 벡터 형태: {w2v_vectors.shape}")

### 4.1 고급 임베딩 방법 (Advanced Embedding Methods)

이제 한국어에 특화된 고급 임베딩 방법을 사용하여 뉴스 텍스트를 벡터화해 보겠습니다.

Now, let's vectorize news text using advanced embedding methods specialized for Korean language.

In [ ]:
# 한국어 임베딩 향상 모듈 임포트
from src.models.korean_embedding_enhancer import KoreanEmbeddingEnhancer

# 한국어 임베딩 향상기 초기화
embedding_enhancer = KoreanEmbeddingEnhancer(
    use_kobert=True,
    use_klue_roberta=True,
    use_kosimcse=True,
    use_bge_korean=True,
    cache_embeddings=True
)

# 사용 가능한 모델 확인
available_models = embedding_enhancer.get_available_models()
print(f"사용 가능한 모델: {available_models}")

# 샘플 텍스트에 대한 임베딩 생성
sample_texts = news_df['Title'].head(3).tolist()

for model in available_models[:2]:  # 처음 두 모델만 시연
    print(f"\n{model} 모델을 사용한 임베딩:")
    for text in sample_texts:
        embedding = embedding_enhancer.get_embedding(text, model)
        print(f"텍스트: {text[:30]}...")
        print(f"임베딩 차원: {embedding.shape}")
        print(f"임베딩 샘플: {embedding[:5]}...")

In [ ]:
# 앙상블 임베딩 생성
print("앙상블 임베딩 생성 중...")
ensemble_embeddings = embedding_enhancer.get_ensemble_embeddings(
    sample_texts,
    models=available_models[:2],  # 처음 두 모델만 사용
    weights=[0.6, 0.4]  # 가중치 설정
)

print(f"앙상블 임베딩 형태: {ensemble_embeddings.shape}")
print(f"앙상블 임베딩 샘플:\n{ensemble_embeddings[0][:5]}...")

# 임베딩 간 유사도 계산
from sklearn.metrics.pairwise import cosine_similarity

similarity_matrix = cosine_similarity(ensemble_embeddings)
print("\n임베딩 간 유사도 행렬:")
print(similarity_matrix)

## 5. 뉴스 클러스터링 (News Clustering)

벡터화된 뉴스를 클러스터링하여 유사한 뉴스 그룹을 식별합니다.

We cluster the vectorized news to identify groups of similar news.

In [ ]:
# 뉴스 클러스터링 모듈 임포트
from src.models.news_clustering import NewsClustering

# 클러스터링 모델 초기화
clustering_model = NewsClustering(
    method='kmeans',
    n_clusters=5,
    random_state=42
)

# 클러스터링 수행
print("TF-IDF 벡터를 사용한 클러스터링 수행 중...")
tfidf_clusters = clustering_model.cluster(tfidf_vectors)

print("Word2Vec 벡터를 사용한 클러스터링 수행 중...")
w2v_clusters = clustering_model.cluster(w2v_vectors)

# 클러스터링 결과 저장
news_df['tfidf_cluster'] = tfidf_clusters
news_df['w2v_cluster'] = w2v_clusters

# 클러스터링 결과 확인
print("\nTF-IDF 클러스터 분포:")
print(news_df['tfidf_cluster'].value_counts())

print("\nWord2Vec 클러스터 분포:")
print(news_df['w2v_cluster'].value_counts())

In [ ]:
# 클러스터 시각화
from sklearn.decomposition import PCA

# PCA를 사용하여 2차원으로 차원 축소
pca = PCA(n_components=2)
tfidf_pca = pca.fit_transform(tfidf_vectors)

# 클러스터 시각화
plt.figure(figsize=(12, 8))
for cluster in range(clustering_model.n_clusters):
    mask = tfidf_clusters == cluster
    plt.scatter(tfidf_pca[mask, 0], tfidf_pca[mask, 1], label=f'Cluster {cluster}', alpha=0.7)

plt.title('TF-IDF 벡터를 사용한 뉴스 클러스터링 결과')
plt.xlabel('PCA Component 1')
plt.ylabel('PCA Component 2')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

### 5.1 고급 클러스터링 방법 (Advanced Clustering Methods)

이제 고급 임베딩을 사용하여 뉴스 클러스터링을 수행해 보겠습니다.

Now, let's perform news clustering using advanced embeddings.

In [ ]:
# 샘플 데이터에 대한 임베딩 생성
sample_df = news_df.head(50)  # 처리 시간 단축을 위해 일부만 사용

# 데이터프레임 벡터화
print("KoBERT를 사용한 벡터화 수행 중...")
kobert_vectors = embedding_enhancer.vectorize_dataframe(
    sample_df,
    text_col='Title',
    content_col='Body',
    use_content=True,
    model='kobert',
    batch_size=10
)

# 앙상블 벡터화
print("앙상블 벡터화 수행 중...")
ensemble_vectors = embedding_enhancer.vectorize_dataframe_ensemble(
    sample_df,
    text_col='Title',
    content_col='Body',
    use_content=True,
    models=available_models[:2],
    weights=[0.6, 0.4],
    batch_size=10
)

print(f"KoBERT 벡터 형태: {kobert_vectors.shape}")
print(f"앙상블 벡터 형태: {ensemble_vectors.shape}")

In [ ]:
# 고급 클러스터링 수행
from sklearn.cluster import AgglomerativeClustering

# 계층적 클러스터링 모델 초기화
hierarchical_clustering = AgglomerativeClustering(
    n_clusters=5,
    affinity='cosine',
    linkage='average'
)

# 클러스터링 수행
kobert_clusters = hierarchical_clustering.fit_predict(kobert_vectors)
ensemble_clusters = hierarchical_clustering.fit_predict(ensemble_vectors)

# 클러스터링 결과 저장
sample_df['kobert_cluster'] = kobert_clusters
sample_df['ensemble_cluster'] = ensemble_clusters

# 클러스터링 결과 확인
print("\nKoBERT 클러스터 분포:")
print(sample_df['kobert_cluster'].value_counts())

print("\n앙상블 클러스터 분포:")
print(sample_df['ensemble_cluster'].value_counts())

In [ ]:
# 클러스터 시각화
# PCA를 사용하여 2차원으로 차원 축소
pca = PCA(n_components=2)
ensemble_pca = pca.fit_transform(ensemble_vectors)

# 클러스터 시각화
plt.figure(figsize=(12, 8))
for cluster in range(5):
    mask = ensemble_clusters == cluster
    plt.scatter(ensemble_pca[mask, 0], ensemble_pca[mask, 1], label=f'Cluster {cluster}', alpha=0.7)

plt.title('앙상블 임베딩을 사용한 뉴스 클러스터링 결과')
plt.xlabel('PCA Component 1')
plt.ylabel('PCA Component 2')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 6. 주식 영향 분석 (Stock Impact Analysis)

뉴스가 주식 가격에 미치는 영향을 분석합니다.

We analyze the impact of news on stock prices.

In [ ]:
# 주식 영향 분석 모듈 임포트
from src.models.stock_impact_analyzer import StockImpactAnalyzer

# 주식 영향 분석기 초기화
impact_analyzer = StockImpactAnalyzer(
    lookback_window=3,  # 3일 전부터의 데이터 사용
    lookahead_window=3,  # 3일 후까지의 데이터 사용
    impact_threshold=0.5  # 영향력 임계값
)

# 주식 영향 분석 수행
print("주식 영향 분석 수행 중...")
impact_scores = impact_analyzer.analyze_impact(
    news_df,
    stock_df,
    news_date_col='datetime',
    stock_date_col='datetime',
    price_col='End'
)

# 영향 점수 저장
news_df['impact_score'] = impact_scores

# 영향 점수 분포 확인
plt.figure(figsize=(10, 6))
plt.hist(news_df['impact_score'].dropna(), bins=20, alpha=0.7)
plt.title('뉴스 영향 점수 분포')
plt.xlabel('영향 점수')
plt.ylabel('뉴스 수')
plt.grid(True, alpha=0.3)
plt.show()

# 영향력이 큰 뉴스 확인
high_impact_news = news_df.sort_values('impact_score', ascending=False).head(5)
print("\n영향력이 큰 뉴스 Top 5:")
for idx, row in high_impact_news.iterrows():
    print(f"제목: {row['Title']}")
    print(f"날짜: {row['Date']}")
    print(f"영향 점수: {row['impact_score']:.4f}")
    print("---")

### 6.1 고급 주식 영향 분석 (Advanced Stock Impact Analysis)

이제 다양한 임베딩 모델을 사용한 고급 주식 영향 분석 방법을 시연합니다.

Now, we demonstrate advanced stock impact analysis methods using various embedding models.

In [ ]:
# 고급 점수 계산 방법 모듈 임포트
from src.models.advanced_scoring_methods import AdvancedScoringMethods

# 고급 점수 계산 방법 초기화
scoring_methods = AdvancedScoringMethods(
    use_kosimcse=True,
    use_bge_korean=True,
    cache_embeddings=True
)

# 감성 점수 계산 (간단한 방법으로 대체)
def calculate_simple_sentiment(text):
    positive_words = ['상승', '급등', '호조', '성장', '개선', '흑자', '최대', '신기록']
    negative_words = ['하락', '급락', '부진', '감소', '악화', '적자', '손실', '하향']
    
    if not isinstance(text, str):
        return 0.0
        
    pos_count = sum(1 for word in positive_words if word in text)
    neg_count = sum(1 for word in negative_words if word in text)
    
    if pos_count + neg_count == 0:
        return 0.0
    
    return (pos_count - neg_count) / (pos_count + neg_count)

# 샘플 데이터에 감성 점수 추가
sample_df['sentiment_score'] = sample_df['Title'].apply(calculate_simple_sentiment)

# 주가 변동 계산 (간단한 방법으로 대체)
sample_df['price_change_pct_1d'] = 0.5  # 예시 값

# 앙상블 영향 점수 계산
print("앙상블 영향 점수 계산 중...")
impact_results = scoring_methods.calculate_ensemble_impact_scores(
    sample_df,
    text_col='Title',
    sentiment_col='sentiment_score',
    price_change_col='price_change_pct_1d',
    models=['kosimcse', 'bge_korean'],
    weights=[0.6, 0.4]
)

# 결과 확인
print("\n앙상블 영향 점수 결과:")
impact_results[['Title', 'sentiment_score', 'impact_kosimcse', 'impact_bge_korean', 'impact_ensemble']].head()

In [ ]:
# 모델 비교 시각화
scoring_methods.visualize_model_comparison(
    impact_results,
    models=['kosimcse', 'bge_korean']
)

# 앙상블 가중치 시각화
scoring_methods.visualize_ensemble_weights(
    weights=[0.6, 0.4],
    models=['kosimcse', 'bge_korean']
)

## 7. 한국어 텍스트 분석 (Korean Text Analysis)

한국어 금융 텍스트에 대한 고급 감성 분석을 수행합니다.

We perform advanced sentiment analysis on Korean financial text.

In [ ]:
# 한국어 텍스트 분석 모듈 임포트
from src.models.korean_text_analyzer import KoreanFinancialTextAnalyzer

# 한국어 텍스트 분석기 초기화
text_analyzer = KoreanFinancialTextAnalyzer(
    use_finbert=True,
    use_advanced_embeddings=True,
    cache_results=True
)

# 샘플 텍스트 분석
sample_texts = [
    "삼성전자 주가 상승, 실적 개선 기대감에 투자자들 관심 집중",
    "삼성전자 주가 하락, 실적 부진 우려에 투자자들 매도세 확대",
    "삼성전자, 신제품 출시 계획 발표. 시장 반응은 중립적"
]

for text in sample_texts:
    result = text_analyzer.analyze_text(text)
    print(f"텍스트: {text}")
    print(f"감성 점수: {result['sentiment_score']:.4f} ({result['sentiment_label']})")
    print(f"신뢰도: {result['confidence']:.4f}")
    print("---")

In [ ]:
# 배치 분석
batch_results = text_analyzer.analyze_batch(sample_texts)

# 결과 시각화
plt.figure(figsize=(10, 6))
plt.bar(range(len(batch_results)), [r['sentiment_score'] for r in batch_results], alpha=0.7)
plt.xticks(range(len(batch_results)), [f"Text {i+1}" for i in range(len(batch_results))])
plt.axhline(y=0, color='r', linestyle='-', alpha=0.3)
plt.title('텍스트 감성 점수')
plt.ylabel('감성 점수 (-1 ~ 1)')
plt.grid(True, alpha=0.3)
plt.show()

## 8. 뉴스 추천 (News Recommendation)

사용자에게 관련성 높은 뉴스를 추천합니다.

We recommend relevant news to users.

In [ ]:
# 뉴스 추천 모듈 임포트
from src.models.news_recommender import NewsRecommender

# 뉴스 추천기 초기화
recommender = NewsRecommender(
    cf_weight=0.3,  # 협업 필터링 가중치
    cbf_weight=0.3,  # 콘텐츠 기반 필터링 가중치
    si_weight=0.2,   # 소셜 영향력 가중치
    latest_weight=0.1, # 최신성 가중치
    stock_impact_weight=0.1 # 주식 영향력 가중치
)

# 추천 수행
print("뉴스 추천 수행 중...")
recommended_news = recommender.recommend(
    news_df,
    user_id=None,  # 비개인화 추천
    vectors=tfidf_vectors,
    top_n=5
)

# 추천 결과 확인
print("\n추천 뉴스:")
for idx, row in recommended_news.iterrows():
    print(f"제목: {row['Title']}")
    print(f"날짜: {row['Date']}")
    print(f"추천 점수: {row['recommendation_score']:.4f}")
    print("---")

### 8.1 고급 뉴스 추천 (Advanced News Recommendation)

이제 고급 임베딩과 다양한 점수 계산 방법을 사용한 뉴스 추천을 시연합니다.

Now, we demonstrate news recommendation using advanced embeddings and various scoring methods.

In [ ]:
# 고급 임베딩을 사용한 추천
# 샘플 데이터에 대한 앙상블 임베딩 사용
print("고급 임베딩을 사용한 추천 수행 중...")
advanced_recommended_news = recommender.recommend(
    sample_df,
    user_id=None,  # 비개인화 추천
    vectors=ensemble_vectors,
    top_n=3,
    stock_impact_col='impact_ensemble'  # 앙상블 영향 점수 사용
)

# 추천 결과 확인
print("\n고급 임베딩을 사용한 추천 뉴스:")
for idx, row in advanced_recommended_news.iterrows():
    print(f"제목: {row['Title']}")
    print(f"날짜: {row['datetime']}")
    print(f"추천 점수: {row['recommendation_score']:.4f}")
    print("---")

In [ ]:
# 다양한 가중치 조합 실험
weight_combinations = [
    {'cf': 0.2, 'cbf': 0.2, 'si': 0.2, 'latest': 0.2, 'stock_impact': 0.2},  # 균등 가중치
    {'cf': 0.1, 'cbf': 0.1, 'si': 0.1, 'latest': 0.1, 'stock_impact': 0.6},  # 주식 영향력 중심
    {'cf': 0.1, 'cbf': 0.6, 'si': 0.1, 'latest': 0.1, 'stock_impact': 0.1},  # 콘텐츠 중심
    {'cf': 0.1, 'cbf': 0.1, 'si': 0.1, 'latest': 0.6, 'stock_impact': 0.1}   # 최신성 중심
]

# 각 가중치 조합에 대한 추천 결과 비교
for i, weights in enumerate(weight_combinations):
    # 추천기 초기화
    custom_recommender = NewsRecommender(
        cf_weight=weights['cf'],
        cbf_weight=weights['cbf'],
        si_weight=weights['si'],
        latest_weight=weights['latest'],
        stock_impact_weight=weights['stock_impact']
    )
    
    # 추천 수행
    recommended = custom_recommender.recommend(
        sample_df,
        user_id=None,
        vectors=ensemble_vectors,
        top_n=1,
        stock_impact_col='impact_ensemble'
    )
    
    # 결과 출력
    print(f"\n가중치 조합 {i+1}:")
    print(f"CF: {weights['cf']}, CBF: {weights['cbf']}, SI: {weights['si']}, Latest: {weights['latest']}, Stock Impact: {weights['stock_impact']}")
    print(f"추천 뉴스: {recommended['Title'].iloc[0]}")
    print(f"추천 점수: {recommended['recommendation_score'].iloc[0]:.4f}")

## 9. 실험적 모델 (Experimental Models)

다양한 실험적 모델과 기법을 시연합니다.

We demonstrate various experimental models and techniques.

In [ ]:
# 다양한 임베딩 모델 비교
from sklearn.metrics.pairwise import cosine_similarity

# 샘플 텍스트
comparison_texts = [
    "삼성전자 신제품 출시, 시장 반응 긍정적",
    "삼성전자 신형 스마트폰 공개, 소비자들 호평",
    "애플 아이폰 신모델 발표, 삼성전자 경쟁 심화",
    "반도체 시장 침체, 삼성전자 실적 우려"
]

# 각 모델별 임베딩 생성
model_embeddings = {}
available_models = embedding_enhancer.get_available_models()

for model in available_models[:3]:  # 처음 세 모델만 비교
    print(f"{model} 모델 임베딩 생성 중...")
    embeddings = [embedding_enhancer.get_embedding(text, model) for text in comparison_texts]
    model_embeddings[model] = np.array(embeddings)

# 앙상블 임베딩 생성
print("앙상블 임베딩 생성 중...")
ensemble_embeddings = embedding_enhancer.get_ensemble_embeddings(
    comparison_texts,
    models=available_models[:3],
    weights=[0.4, 0.3, 0.3]
)
model_embeddings['ensemble'] = ensemble_embeddings

# 각 모델별 유사도 행렬 계산 및 시각화
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
axes = axes.flatten()

for i, (model, embeddings) in enumerate(model_embeddings.items()):
    # 유사도 행렬 계산
    similarity_matrix = cosine_similarity(embeddings)
    
    # 히트맵 시각화
    sns.heatmap(similarity_matrix, annot=True, cmap='coolwarm', vmin=0, vmax=1, ax=axes[i])
    axes[i].set_title(f'{model} 모델 유사도 행렬')
    axes[i].set_xticklabels([f'Text {i+1}' for i in range(len(comparison_texts))], rotation=45)
    axes[i].set_yticklabels([f'Text {i+1}' for i in range(len(comparison_texts))], rotation=0)

plt.tight_layout()
plt.show()

In [ ]:
# 다양한 클러스터링 알고리즘 비교
from sklearn.cluster import KMeans, DBSCAN, SpectralClustering
from sklearn.metrics import silhouette_score

# 임베딩 데이터 준비 (앙상블 임베딩 사용)
X = ensemble_embeddings

# 클러스터링 알고리즘 정의
clustering_algorithms = {
    'KMeans': KMeans(n_clusters=3, random_state=42),
    'DBSCAN': DBSCAN(eps=0.5, min_samples=2),
    'SpectralClustering': SpectralClustering(n_clusters=3, random_state=42)
}

# 각 알고리즘 실행 및 결과 비교
results = {}
for name, algorithm in clustering_algorithms.items():
    print(f"{name} 알고리즘 실행 중...")
    labels = algorithm.fit_predict(X)
    results[name] = labels
    
    # 실루엣 점수 계산 (DBSCAN은 클러스터 수가 가변적이므로 제외)
    if name != 'DBSCAN':
        score = silhouette_score(X, labels)
        print(f"{name} 실루엣 점수: {score:.4f}")
    
    # 클러스터 분포 출력
    unique_labels = np.unique(labels)
    print(f"클러스터 수: {len(unique_labels)}")
    for label in unique_labels:
        print(f"클러스터 {label}: {np.sum(labels == label)}개 샘플")
    print("---")

## 10. 시스템 통합 (System Integration)

전체 시스템을 통합하여 완전한 파이프라인을 시연합니다.

We integrate the entire system to demonstrate a complete pipeline.

In [ ]:
# 전체 시스템 통합 파이프라인
def run_clear_pipeline(news_data, stock_data, use_advanced_models=False):
    print("CLEAR 시스템 파이프라인 실행 중...")
    
    # 1. 데이터 전처리
    print("1. 데이터 전처리 중...")
    # 날짜 형식 변환
    news_data['datetime'] = pd.to_datetime(news_data['Date'].apply(convert_date_format))
    news_data = news_data.sort_values('datetime')
    
    stock_data['datetime'] = pd.to_datetime(stock_data['Date'] + ' ' + stock_data['Time'])
    stock_data = stock_data.sort_values('datetime')
    
    # 주식 가격 변동 계산
    stock_data['price_change'] = stock_data['End'] - stock_data['Start']
    stock_data['price_change_pct'] = (stock_data['price_change'] / stock_data['Start']) * 100
    
    # 2. 텍스트 전처리
    print("2. 텍스트 전처리 중...")
    preprocessor = TextPreprocessor(language='korean', remove_stopwords=True)
    news_data['processed_title'] = news_data['Title'].apply(preprocessor.preprocess_text)
    news_data['processed_body'] = news_data['Body'].apply(
        lambda x: preprocessor.preprocess_text(x[:500]) if isinstance(x, str) else ""
    )
    
    # 3. 뉴스 벡터화
    print("3. 뉴스 벡터화 중...")
    if use_advanced_models:
        # 고급 임베딩 사용
        embedding_enhancer = KoreanEmbeddingEnhancer(
            use_kobert=True,
            use_klue_roberta=True
        )
        vectors = embedding_enhancer.vectorize_dataframe_ensemble(
            news_data.head(50),  # 시연을 위해 일부만 사용
            text_col='Title',
            content_col='Body',
            use_content=True,
            models=['kobert', 'klue_roberta'],
            weights=[0.6, 0.4],
            batch_size=10
        )
        news_subset = news_data.head(50)
    else:
        # 기본 벡터화 방법 사용
        vectorizer = NewsVectorizer(method='tfidf', title_weight=0.7, content_weight=0.3)
        vectors = vectorizer.vectorize_dataframe(
            news_data,
            title_col='processed_title',
            content_col='processed_body'
        )
        news_subset = news_data
    
    # 4. 뉴스 클러스터링
    print("4. 뉴스 클러스터링 중...")
    clustering_model = NewsClustering(method='kmeans', n_clusters=5)
    clusters = clustering_model.cluster(vectors)
    news_subset['cluster'] = clusters
    
    # 5. 주식 영향 분석
    print("5. 주식 영향 분석 중...")
    if use_advanced_models:
        # 고급 점수 계산 방법 사용
        # 감성 점수 계산 (간단한 방법으로 대체)
        news_subset['sentiment_score'] = news_subset['Title'].apply(calculate_simple_sentiment)
        news_subset['price_change_pct_1d'] = 0.5  # 예시 값
        
        scoring_methods = AdvancedScoringMethods(use_kosimcse=True)
        impact_results = scoring_methods.calculate_ensemble_impact_scores(
            news_subset,
            text_col='Title',
            sentiment_col='sentiment_score',
            price_change_col='price_change_pct_1d'
        )
        news_subset = impact_results
        impact_col = 'impact_ensemble'
    else:
        # 기본 영향 분석 방법 사용
        impact_analyzer = StockImpactAnalyzer(lookback_window=3, lookahead_window=3)
        impact_scores = impact_analyzer.analyze_impact(
            news_subset,
            stock_data,
            news_date_col='datetime',
            stock_date_col='datetime',
            price_col='End'
        )
        news_subset['impact_score'] = impact_scores
        impact_col = 'impact_score'
    
    # 6. 뉴스 추천
    print("6. 뉴스 추천 중...")
    recommender = NewsRecommender(
        cf_weight=0.2,
        cbf_weight=0.3,
        si_weight=0.2,
        latest_weight=0.1,
        stock_impact_weight=0.2
    )
    recommended_news = recommender.recommend(
        news_subset,
        user_id=None,  # 비개인화 추천
        vectors=vectors,
        top_n=5,
        stock_impact_col=impact_col
    )
    
    print("\nCLEAR 시스템 파이프라인 실행 완료!")
    return recommended_news

# 기본 모델로 파이프라인 실행
print("기본 모델로 파이프라인 실행:")
basic_recommendations = run_clear_pipeline(news_df.copy(), stock_df.copy(), use_advanced_models=False)

print("\n기본 모델 추천 결과:")
for idx, row in basic_recommendations.head(3).iterrows():
    print(f"제목: {row['Title']}")
    print(f"추천 점수: {row['recommendation_score']:.4f}")
    print("---")

In [ ]:
# 고급 모델로 파이프라인 실행
print("고급 모델로 파이프라인 실행:")
advanced_recommendations = run_clear_pipeline(news_df.copy(), stock_df.copy(), use_advanced_models=True)

print("\n고급 모델 추천 결과:")
for idx, row in advanced_recommendations.head(3).iterrows():
    print(f"제목: {row['Title']}")
    print(f"추천 점수: {row['recommendation_score']:.4f}")
    print("---")

## 결론 (Conclusion)

이 노트북에서는 NAVER의 AiRS 알고리즘을 기반으로 한 뉴스 추천 및 주식 영향 분석 시스템인 CLEAR의 기능을 시연했습니다. 시스템은 다음과 같은 주요 구성 요소로 이루어져 있습니다:

1. 데이터 처리 (뉴스 및 주식 데이터)
2. 텍스트 전처리
3. 뉴스 벡터화 (기본 및 고급 임베딩 방법)
4. 뉴스 클러스터링
5. 주식 영향 분석 (기본 및 고급 점수 계산 방법)
6. 뉴스 추천

또한 다양한 한국어 임베딩 모델과 점수 계산 방법을 비교하고, 이를 통합하여 완전한 파이프라인을 구현했습니다.

In this notebook, we demonstrated the functionality of CLEAR, a news recommendation and stock impact analysis system based on NAVER's AiRS algorithm. The system consists of the following main components:

1. Data processing (news and stock data)
2. Text preprocessing
3. News vectorization (basic and advanced embedding methods)
4. News clustering
5. Stock impact analysis (basic and advanced scoring methods)
6. News recommendation

We also compared various Korean embedding models and scoring methods, and integrated them to implement a complete pipeline.